# Data Privacy Final Project
### Sam Clark & Josh Childs

For this project, we've decided to compare the accuracy of several normal Convolutional Neural Networks to their counter parts that will use differential privacy. We will be using the MNIST dataset with the tensflow library.  

In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from ipywidgets import IntProgress
from sklearn.metrics import classification_report
import tensorflow_privacy

tf.enable_v2_behavior()

## Load MNIST Data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

## Preprocess Data

In [3]:
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Build Models

In [5]:
layers = [
  tf.keras.layers.Conv2D(28, kernel_size=(3,3), input_shape=input_shape),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation=tf.nn.relu),  
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(10,activation=tf.nn.softmax)
]

model1 = tf.keras.models.Sequential(layers)
model1.summary()

models = [model1]

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 28)        280       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 28)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4732)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               605824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 607,394
Trainable params: 607,394
Non-trainable params: 0
________________________________________________

## Train & Save

In [6]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

for model in models:
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

    model.fit(x=x_train,y=y_train, epochs=1, callbacks = [callback])
    model.save("models/" + str(models.index(model) + 1 ))

1875/1875 [==============================] - 24s 13ms/step - loss: 0.2375 - accuracy: 0.9289
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: models/1/assets


## Evaluate Models

In [7]:
metrics = {}

for model in models:
    metrics["model" + str(models.index(model) + 1)] = model.evaluate(x_test,y_test)
    

313/313 [==============================] - 1s 3ms/step - loss: 0.0828 - accuracy: 0.9754


In [8]:
metrics

{'model1': [0.0828162357211113, 0.9753999710083008]}

## Add Differential Privacy
#### Source: (http://www.cleverhans.io/privacy/2019/03/26/machine-learning-with-differential-privacy-in-tensorflow.html)

In [9]:
dp_optimizer = tensorflow_privacy.privacy.optimizers.dp_optimizer.DPGradientDescentGaussianOptimizer(
    l2_norm_clip=1.5,
    noise_multiplier=1.3,
    num_microbatches=256,
    learning_rate=.25)
dp_optimizer.compute_gradients()

TypeError: compute_gradients() missing 2 required positional arguments: 'loss' and 'var_list'

In [69]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)


for model in models:
    model.compile(optimizer=dp_optimizer, 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])

    model.fit(x=x_train,y=y_train, epochs=1, callbacks = [callback])
    model.save("dp_models/" + str(models.index(model) + 1 ))

AssertionError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:806 train_function  *
        return step_function(self, iterator)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:796 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute